In [97]:
import requests
import pandas as pd
import numpy as np
import json
import folium

api_key = "tL6lWEb1UKgKz1n2FmKGhXFheZNPQdaW"

In [98]:


def initialise_map(api_key=api_key, location=[55.9312, 12.29651], zoom=14, style = "main"):
    maps_url = "http://a.api.tomtom.com/map/1/tile/basic/"+style+"/{z}/{x}/{y}.png?tileSize=512&key="
    TomTom_map = folium.Map(
    location = location,
    zoom_start = zoom,
    tiles = str(maps_url + api_key),
    attr = 'TomTom')
    return TomTom_map

TomTom_map = initialise_map()

In [99]:

start_address = "Slangerupgade 22a Hillerød 3400"
end_address = "Nørgaardvej 30 Lyngby 2800"

def get_depature_and_destination_coords(start_address, end_address):
    start_address_list = start_address.split()
    end_address_list = end_address.split()

    geolocation_endpoint = f'https://api.tomtom.com/search/2/structuredGeocode.json?&key={api_key}'

    params1 = {
        "countryCode": "DK",
        "streetName": start_address_list[0],
        "streetNumber": start_address_list[1],
        "municipality": start_address_list[2],
        "postalCode": start_address_list[3],
        "limit": "1"
        }
    
    params2 = {
        "countryCode": "DK",
        "streetName": end_address_list[0],
        "streetNumber": end_address_list[1],
        "municipality": end_address_list[2],
        "postalCode": end_address_list[3],
        "limit": "1"
        }

    response1 = requests.get(geolocation_endpoint, params = params1)
    start_address_data = response1.json()
    
    response2 = requests.get(geolocation_endpoint, params = params2)
    end_address_data = response2.json()
    

    start_entry_point = start_address_data["results"][0]["entryPoints"][0]["position"]
    
    end_entry_point = end_address_data["results"][0]["entryPoints"][0]["position"]
    
    return start_entry_point, end_entry_point;


print(get_depature_and_destination_coords(start_address, end_address))

({'lat': 55.93125, 'lon': 12.29628}, {'lat': 55.77165, 'lon': 12.50561})


In [101]:
#define parameteres for route

#endpoint = "https://api.tomtom.com/routing/1/calculateRoute/{}/:{}/json"

start = '55.93125,12.29628' # latitude,longitude of the start location
end = '55.77018,12.51145' # latitude,longitude of the end location

def get_fastes_route(start_coords, end_coords):
    

    route_endpoint = f'https://api.tomtom.com/routing/1/calculateRoute/{start_coords}:{end_coords}/json?&key={api_key}'


    params = {
        "traffic": "false",
        "travelMode": "car",
        "routeType": "fastest",
        "instructionsType": "text",
        "language": "en-US",
        "computeBestOrder": "false",
        }

    response = requests.get(route_endpoint, params=params)
    data = response.json()


    route_geo = data["routes"][0]["legs"][0]["points"]

    points = [(c['latitude'], c['longitude']) for c in route_geo]

    folium.PolyLine(locations=points, color='red').add_to(TomTom_map)
    
    return